<a href="https://colab.research.google.com/github/Emmahgithub/Analisis-Numerico-2025-1/blob/main/21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Tarea 1 Analisis Numerico</i></h1>
  </font>
  <font color="Blue" face="Comic Sans MS,arial">
  <h5 align="center"><i>MARTINEZ ROSAS ZAYDE YAMILE</i></h5>
  <h5 align="center"><i>CAMACHO MARIN ANA KAREN</i></h5>
  <h5 align="center"><i>LÓPEZ AGUIRRE ROBERTO OCELOTZIN</i></h5>
  <h5 align="center"><i>VARGAS BAUTISTA EMMANUEL</i></h5>
  </font>

In [ ]:
#necesitamos estas funciones (Ayudante code)
import numpy as np

def SustitucionDelante(L, b):
    n = len(b)
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    return y

def SustitucionAtras(U, y):
    n = len(y)
    x = np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

In [ ]:

# Factorización LU sin pivoteo (ayudante)
def LU(A):
  U=np.copy(A)
  L=np.eye(A.shape[0])

  for i in range(A.shape[0]):
    Li=np.eye(A.shape[0])
    for j in range(i+1,A.shape[0]):
      Li[j,i]=-U[j,i]/U[i,i]
      L[j,i]=U[j,i]/U[i,i]
    U=Li@U
  return L,U

def SolverLU(A, b):
    L, U = LU(A)
    y = SustitucionDelante(L, b)
    x = SustitucionAtras(U, y)
    return x

In [ ]:

# Factorizacion LU con pivoteo parcial (parcial ayudante)
def Permutaciones(A,b):
  U=np.copy(A)
  x=np.copy(b)
  P=np.eye(A.shape[0])

  for j in range(A.shape[0]):
    k=np.argmax(np.abs(A[j:,j]))+j
    U[[j,k]]=U[[k,j]]
    P[[j,k]]=P[[k,j]]
    x[[j,k]]=x[[k,j]]

  return P,U,x



def Solver_LU_Pivot_Partial(A,b):
    P, Q, Ap, bp = Permutaciones(A,b)  # Ahora capturamos los 4 valores
    x = SolverLU(Ap, bp)
    x = Q @ x  # Aplicamos la permutación de columnas
    return x


In [ ]:
# Factorización LU con pivoteo (ayudante)

def Permutaciones(A, b):
    n = A.shape[0]  # Tamaño de la matriz (n x n)
    U = np.copy(A)  # Copia de la matriz A para no modificarla directamente
    x = np.copy(b)  # Copia del vector b
    P = np.eye(n)   # Matriz de permutación de filas (inicialmente identidad)
    Q = np.eye(n)   # Matriz de permutación de columnas (inicialmente identidad)

    for k in range(n - 1):
        # --- PASO 1: Encontrar el pivote máximo en la submatriz U[k:, k:] ---
        # np.abs(U[k:, k:]): Valores absolutos de la submatriz desde (k,k) hasta el final
        # np.argmax(...): Índice lineal del elemento con mayor valor absoluto (en arreglo aplanado)
        # np.unravel_index(...): Convierte el índice lineal a coordenadas (fila, columna) en la submatriz
        max_row, max_col = np.unravel_index(np.argmax(np.abs(U[k:, k:])), (n - k, n - k))

        # Ajustar índices para referenciar la posición en la matriz original U (no solo la submatriz)
        max_row += k
        max_col += k

        # --- PASO 2: Intercambiar filas para llevar el pivote a la posición (k,k) ---
        # Intercambia filas en U
        U[[k, max_row]] = U[[max_row, k]]
        # Intercambia filas en la matriz de permutación P
        P[[k, max_row]] = P[[max_row, k]]
        # Intercambia elementos en el vector x
        x[[k, max_row]] = x[[max_row, k]]

        # --- PASO 3: Intercambiar columnas para optimizar la factorización ---
        # Intercambia columnas en U
        U[:, [k, max_col]] = U[:, [max_col, k]]
        # Intercambia columnas en la matriz de permutación Q
        Q[:, [k, max_col]] = Q[:, [max_col, k]]

    return P, Q, U, x  # Devuelve: Matrices de permutación, matriz triangular, y vector modificado


def Solver_LU_Pivot_Total(A,b):
    P,Q,A_g,b_g=Permutaciones(A,b)
    x=SolverLU(A_g,b_g)
    x=Q@x

    return x



In [ ]:
  #matrices solionar con cada uno de los metodos de factorizacion LU
# (a)
A1 = np.array([[4, -1, 3],
               [-8, 4, -7],
               [12, 1, 8]], dtype=float)
b1 = np.array([-8, 19, -19], dtype=float)

# (b)
A2 = np.array([[1, 4, -2, 1],
               [-2, -4, -3, 1],
               [1, 16, -17, 9],
               [2, 4, -9, -3]], dtype=float)
b2 = np.array([3.5, -2.5, 15, 10.5], dtype=float)

# (c)
A3 = np.array([[4, 5, -1, 2, 3],
               [12, 13, 0, 10, 3],
               [-8, -8, 5, -11, 4],
               [16, 18, -7, 20, 4],
               [-4, -9, 31, -31, -1]], dtype=float)
b3 = np.array([34, 93, -33, 131, -58], dtype=float)


#implementacion para conocer y ver las difencias entre cada uno de los metodos

# Resolver con LU sin pivoteo
print("=== Método LU sin pivoteo ===")
print("Sistema (a):", A1 @ SolverLU(A1, b1))
print("Sistema (b):", A2 @ SolverLU(A2, b2))
print("Sistema (c):", A3 @ SolverLU(A3, b3))

# Resolver con LU con pivoteo parcial
print("\n=== Método LU con pivoteo parcial ===")
print("Sistema (a):", A1 @ Solver_LU_Pivot_Partial(A1, b1))
print("Sistema (b):", A2 @ Solver_LU_Pivot_Partial(A2, b2))
print("Sistema (c):", A3 @ Solver_LU_Pivot_Partial(A3, b3))

# Resolver con LU con pivoteo total
print("\n=== Método LU con pivoteo total ===")
print("Sistema (a):", A1 @ Solver_LU_Pivot_Total(A1, b1))
print("Sistema (b):", A2 @ Solver_LU_Pivot_Total(A2, b2))
print("Sistema (c):", A3 @ Solver_LU_Pivot_Total(A3, b3))

# Verificación
print("\n=== Verificación ===")
print("A1 @ x =", A1 @ Solver_LU_Pivot_Total(A1, b1))
print("A2 @ x =", A2 @ Solver_LU_Pivot_Total(A2, b2))
print("A3 @ x =", A3 @ Solver_LU_Pivot_Total(A3, b3))


=== Método LU sin pivoteo ===
Sistema (a): [ -8.  19. -19.]
Sistema (b): [ 3.5 -2.5 15.  10.5]
Sistema (c): [ 34.  93. -33. 131. -58.]

=== Método LU con pivoteo parcial ===
Sistema (a): [ -8.  19. -19.]
Sistema (b): [ 3.5 -2.5 15.  10.5]
Sistema (c): [ 34.  93. -33. 131. -58.]

=== Método LU con pivoteo total ===
Sistema (a): [ -8.  19. -19.]
Sistema (b): [ 3.5 -2.5 15.  10.5]
Sistema (c): [ 34.  93. -33. 131. -58.]

=== Verificación ===
A1 @ x = [ -8.  19. -19.]
A2 @ x = [ 3.5 -2.5 15.  10.5]
A3 @ x = [ 34.  93. -33. 131. -58.]
